# Cell Inference Sentimen Terhadap Aplikasi Udemy

**Submission Proyek Analisis Sentimen**

**DBS Foundation Coding Camp**


*   **Nama**            : Roni Antonius Sinabutar
*   **Username**   : aantoniusron
*   **Domisili**  : Cianjur, Jawa Barat
*   **email**      : aantoniusron@gmail.com
*   **Kelas**      : Belajar Pengembangan Machine Learning

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
import nltk
import torch
import random
import pickle
import inflect
import re, string
import unicodedata
import contractions

In [4]:
from pyforest                          import *
from wordcloud                         import WordCloud
from nltk.corpus                       import stopwords
from keras.models                      import Sequential
from nltk.stem.porter                  import PorterStemmer
from bs4                               import BeautifulSoup
from sklearn.feature_extraction.text   import TfidfVectorizer
from sklearn.model_selection           import train_test_split
from nltk.sentiment.vader              import SentimentIntensityAnalyzer
from sklearn.metrics.pairwise          import cosine_similarity, linear_kernel
from sklearn.metrics                   import classification_report, accuracy_score
from keras.layers                      import Embedding, GRU, Dense, Dropout, Flatten
from sklearn.metrics                   import precision_score, recall_score, f1_score
from nltk.tokenize                     import word_tokenize, sent_tokenize, regexp_tokenize
from nltk.stem                         import LancasterStemmer, WordNetLemmatizer, SnowballStemmer

from transformers                      import (Trainer,
                                               AutoTokenizer,
                                               RobertaTokenizer,
                                               TrainingArguments,
                                               DistilBertTokenizer,
                                               RobertaForSequenceClassification,
                                               AutoModelForSequenceClassification,
                                               DistilBertForSequenceClassification)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
twitter_roberta_tokenizer = torch.load('/content/drive/MyDrive/Colab Notebooks/dataset/twitter_roberta_tokenizer.pth')

In [7]:
twitter_roberta_model     = AutoModelForSequenceClassification.from_pretrained(
    'cardiffnlp/twitter-roberta-base-sentiment',
     num_labels=3)


model_state_dict = torch.load('/content/drive/MyDrive/Colab Notebooks/dataset/twitter_roberta_model.pth',
                              map_location=torch.device('cpu'))

twitter_roberta_model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [8]:
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)
    text = re.sub(r'#[A-Za-z0-9]+', '', text)
    text = re.sub(r'RT[\s]', '', text)
    text = re.sub(r"http\S+", '', text)
    text = re.sub(r'[0-9]+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.replace('\n', ' ')
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip()
    return text

def casefoldingText(text):
    text = text.lower()
    return text

def tokenizingText(text):
    text = word_tokenize(text)
    return text

def filteringText(text):
    stop_words_eng = set(stopwords.words('english'))
    filtered_text  = [word for word in text if word.lower() not in stop_words_eng]
    return filtered_text

def stemmingTextSnowball(text):
    snowball = SnowballStemmer('english')
    stemmed_words = [snowball.stem(word) for word in text]
    stemmed_text = ' '.join(stemmed_words)
    return stemmed_text

class TwitterRobertaSentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return 1


In [9]:
def sentimen_udemy(teks):

    kalimat_baru_cleaned     = cleaningText(teks)
    kalimat_baru_casefolded  = casefoldingText(kalimat_baru_cleaned)
    kalimat_baru_tokenized   = tokenizingText(kalimat_baru_casefolded)
    kalimat_baru_filtered    = filteringText(kalimat_baru_tokenized)
    kalimat_baru_final       = stemmingTextSnowball(kalimat_baru_filtered)

    inputs = twitter_roberta_tokenizer(kalimat_baru_final, return_tensors='pt', truncation=True, padding=True)

    with torch.no_grad():
        outputs = twitter_roberta_model(**inputs)

    logits             = outputs.logits
    prediksi_sentimen  = torch.argmax(logits, dim=1)
    label_map          = {2: 'positive', 0: 'negative', 1: 'neutral'}
    predicted_label    = label_map[prediksi_sentimen.item()]

    return predicted_label

In [10]:
teks1     = input("Masukkan kalimat baru: ")
sentimen1 = sentimen_udemy(teks1)
print(f"Sentimen kalimat baru adalah {sentimen1}.")

Masukkan kalimat baru: Udemy has helped me to upskill and reskill in a way that's convenient and affordable. I highly recommend it to anyone looking to improve their career prospects.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Sentimen kalimat baru adalah positive.


In [11]:
teks2     = input("Masukkan kalimat baru: ")
sentimen2 = sentimen_udemy(teks2)
print(f"Sentimen kalimat baru adalah {sentimen2}.")

Masukkan kalimat baru: Udemy is a total waste of time. The courses are poorly made and the instructors are unengaging


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Sentimen kalimat baru adalah negative.


In [12]:
teks3     = input("Masukkan kalimat baru: ")
sentimen3 = sentimen_udemy(teks3)
print(f"Sentimen kalimat baru adalah {sentimen3}.")

Masukkan kalimat baru: I've been using Udemy for a bit now, and it's been a mixed bag


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Sentimen kalimat baru adalah neutral.
